# **Part I. ETL Pipeline for Pre-Processing the Files**

## Pre-processing the files
---

Here we are getting the sample files from `event_data` subdirectory to create an unique `.csv` file to be managed by Apache Cassandra.

#### Importing Python packages

In [1]:
# Importing Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/kellermann/git_repo/sparkify_nosql


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 
---

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing Apache Cassandra code in the cells below
---

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine .
# (loopback IP 127.0.0.1 or hostname localhost)
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries we need a session:
session = cluster.connect()

#### Create Keyspace

In [6]:
# Creating keyspace.
set_ks = 'sparkify_ks'
create_keyspace = """create keyspace if not exists %s with replication = {'class': 'SimpleStrategy', 'replication_factor':1}"""%(set_ks)
session.execute(create_keyspace)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace(set_ks)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4.
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.

## Table #1: artist and song in a session
---
Question to answer:
```Quote
1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4.
```

Here we need to create a table which primary key is composed by `session_id` and `item_in_session`, where `session_id` it's the partition key.
Partitioning by `session_id` guarantees that all records for the same session will be stored in the same partition.

In [8]:
create1 = """create table if not exists session( session_id smallint
                                                ,item_in_session smallint
                                                ,artist_name varchar
                                                ,song_length float
                                                ,song_title varchar
                                                ,primary key(session_id, item_in_session));"""
session.execute(create1)

In [9]:
file = "event_datafile_new.csv"
insert1 = "insert into session(session_id, item_in_session, artist_name,  song_length,  song_title)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        #  0 artist_name
        #  1 user_first_name
        #  2 gender
        #  3 item_in_session
        #  4 user_last_name
        #  5 song_length
        #  6 level 
        #  7 location
        #  8 session_id
        #  9 song_title
        #  10 user_id
        query = insert1 + " values(%s, %s, %s, %s, %s)"""
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))

#### Validating table #1

Here we test by appliying a query that answers question #1.

In [10]:
# Validating:
## select statement for session_id = 338, item_in_session = 4.
select1 = """select artist_name
                  , song_title
                  , song_length
            from session where session_id = 338 and item_in_session = 4;"""

results1 = session.execute(select1)
for row in results1:
    print(f"""
    Artist: {row.artist_name}.
    Song title: {row.song_title}.
    Song length: {row.song_length}.""")


    Artist: Faithless.
    Song title: Music Matters (Mark Knight Dub).
    Song length: 495.30731201171875.


## Table #2: songs played by user and which session was
---
Question to answer:
```Quote
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.
```

Here we need to create a table which primary key is composed by `user_id`, `session_id` and `item_in_session`. Here we need a composed partition key made up `user_id` and `session_id` to guarantee that all records for an user in a session is stored in the same partition. The `ìtem_in_session` is the clustering column that will show us in which order the songs were played by an user. This collumn will appear in ascending sorted order.

In [11]:
create2 = """create table if not exists user_session(user_id  smallint
                                                    ,session_id  smallint
                                                    ,item_in_session smallint
                                                    ,artist_name varchar
                                                    ,song_title  varchar
                                                    ,user_first_name  varchar
                                                    ,user_last_name  varchar
                                                    ,primary key((user_id, session_id), item_in_session));"""

session.execute(create2)

insert2 = "insert into user_session(user_id, session_id, item_in_session, artist_name, song_title, user_first_name, user_last_name)"
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        #  0 artist_name
        #  1 user_first_name
        #  2 gender
        #  3 item_in_session
        #  4 user_last_name
        #  5 song_length
        #  6 level 
        #  7 location
        #  8 session_id
        #  9 song_title
        #  10 user_id
        query = insert2 + " values(%s, %s, %s, %s, %s, %s, %s)"""
        session.execute(query, (int(line[10]),  int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Validating table #2

Here we test by appliying a query that answers question #2.

In [12]:
# Validating:
## select statement for userid = 10, sessionid = 182.
select2 = """select artist_name 
                    ,song_title
                    ,user_first_name
                    ,user_last_name
                    ,user_id
                    ,session_id
                    ,item_in_session
            from user_session where user_id = 10 and session_id = 182;"""

results2 = session.execute(select2)
for row in results2:
    print(f"""
    Artist: {row.artist_name}.
    Song title: {row.song_title}.
    User name: {row.user_first_name} {row.user_last_name}.
    User id: {row.user_id}.
    Session id: {row.session_id}.
    Item in session: {row.item_in_session}.""")


    Artist: Down To The Bone.
    Song title: Keep On Keepin' On.
    User name: Sylvie Cruz.
    User id: 10.
    Session id: 182.
    Item in session: 0.

    Artist: Three Drives.
    Song title: Greece 2000.
    User name: Sylvie Cruz.
    User id: 10.
    Session id: 182.
    Item in session: 1.

    Artist: Sebastien Tellier.
    Song title: Kilometer.
    User name: Sylvie Cruz.
    User id: 10.
    Session id: 182.
    Item in session: 2.

    Artist: Lonnie Gordon.
    Song title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit).
    User name: Sylvie Cruz.
    User id: 10.
    Session id: 182.
    Item in session: 3.


## Table #3: name of an user who played an specific song independently from session or order.
---
Question to answer:
```Quote
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.
```

Here we need to create a table which primary key is composed by `song_title` and `user_id`, where `song_title` is the partition key and `user_id` is the clustering column. By using `song_title` as a partition key we guarantee that all records for a song is stored in the same partition and by using `user_id` as clustering column sort the order at which the users appears by they *id* instead of their names.

In [13]:
create3 = """create table if not exists user_history( song_title  varchar
                                                     ,user_id smallint
                                                     ,user_first_name  varchar
                                                     ,user_last_name  varchar
                                                     ,primary key((song_title), user_id));"""
session.execute(create3)

insert3 = "insert into user_history(song_title, user_id, user_first_name, user_last_name)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        #  0 artist_name
        #  1 user_first_name
        #  2 gender
        #  3 item_in_session
        #  4 user_last_name
        #  5 song_length
        #  6 level 
        #  7 location
        #  8 session_id
        #  9 song_title
        #  10 user_id
        query = insert3 + " values(%s, %s, %s, %s)"""
        session.execute(query, (line[9], int(line[10]), line[1], line[4],))

#### Validating table #3

Here we test by appliying a query that answers question #3.

In [14]:
# Validating:
# select statement to get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.

song_title = 'All Hands Against His Own'

select3 = f"""select user_first_name
                    , user_last_name 
              from user_history 
                    where song_title = '{song_title}';"""

results3 = session.execute(select3)
for row in results3:
    print(f"""User name: {row.user_first_name} {row.user_last_name}.""")

User name: Jacqueline Lynch.
User name: Tegan Levine.
User name: Sara Johnson.


### Drop the tables before closing out the sessions

In [15]:
session.execute('drop table if exists session;')
session.execute('drop table if exists user_session;')
session.execute('drop table if exists user_history;')

### Close the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()

<center>end</center>